In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_selection import RFE

import warnings
import numpy as np
import pandas as pd
import time
from xgboost import XGBRegressor, plot_importance


In [ ]:
train_df = pd.read_csv("../datasets/modified_train_v3.csv")
test_df = pd.read_csv("../datasets/modified_test_v3.csv")
ids = test_df['TRIP_ID']
train_df.head()


In [ ]:
train_df['DURATION'] = train_df['DURATION'] / 15
train_cl = train_df.drop(columns=['TRIP_ID','TIMESTAMP','DATE', 'END_TIME',  'DURATION', 'ORIGIN_CALL',	'ORIGIN_STAND'])
test_cl= test_df.drop(columns=[ 'TRIP_ID','TIMESTAMP','DATE', 'END_TIME', 'ORIGIN_CALL',	'ORIGIN_STAND'])
train_cl.head()

In [ ]:
# values = {'ORIGIN_CALL': -1, 'ORIGIN_STAND': -1}
# train_cl = train_cl.fillna(value=values)
# test_cl = test_cl.fillna(value=values)
test_cl.head()

In [ ]:
train = train_cl


In [ ]:
test = test_cl


In [ ]:
feature_names = list(train.columns)

In [ ]:
y= np.log(train_df['DURATION']) + 1

In [ ]:
Xtr, Xv, ytr, yv = train_test_split(train[feature_names].values, y, test_size=0.2, random_state=1987)
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(test[feature_names].values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

#best
xgb_pars = {'min_child_weight': 10, 'eta': 0.15, 'colsample_bytree': 0.5, 'max_depth': 15, 'subsample': 0.9, 'lambda': 2.0, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}




In [ ]:
model = xgb.train(xgb_pars, dtrain, 10, watchlist, early_stopping_rounds=10,
                  maximize=False, verbose_eval=1)

In [ ]:
logytest = model.predict(dtest)
ytest = (np.exp(logytest) -1 )
print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest*15})
result.to_csv("../datasets/20tree_param_10epoch_xgboost_regression.csv", index=False)

The following section was used to find the best parameters for training.

In [ ]:
FOREVER_COMPUTING_FLAG = True
xgb_pars = []
for MCW in [10, 20, 50, 75, 100]:
    for ETA in [0.05, 0.1, 0.15]:
        for CS in [0.3, 0.4, 0.5]:
            for MD in [6, 8, 10, 12, 15]:
                for SS in [0.5, 0.6, 0.7, 0.8, 0.9]:
                    for LAMBDA in [0.5, 1., 1.5,  2., 3.]:
                        xgb_pars.append({'min_child_weight': MCW, 'eta': ETA, 
                                         'colsample_bytree': CS, 'max_depth': MD,
                                         'subsample': SS, 'lambda': LAMBDA, 
                                         'nthread': -1, 'booster' : 'gbtree', 'eval_metric': 'rmse',
                                         'silent': 1, 'objective': 'reg:linear'})
# count= 0
while FOREVER_COMPUTING_FLAG:
    xgb_par = np.random.choice(xgb_pars, 1)[0]
    print(xgb_par)
    model = xgb.train(xgb_par, dtrain, 10, watchlist, early_stopping_rounds=50,
                      maximize=False, verbose_eval=10)
    # nametouse = "/content/drive/My Drive/CE4032/dataset/" + str(count) + ".csv"
    print('Modeling RMSLE %.5f' % model.best_score)
    # logytest = model.predict(dtest)
    # ytest = (np.exp(logytest) -1 ) * 15
    # print('Test shape OK.') if test.shape[0] == ytest.shape[0] else print('Oops')
    # result= pd.DataFrame({"TRIP_ID": test_df["TRIP_ID"], "TRAVEL_TIME": ytest})
    # result.to_csv(str(nametouse), index=False)
    # count +=1


[9]	train-rmse:0.934293	valid-rmse:0.935502
{'min_child_weight': 50, 'eta': 0.15, 'colsample_bytree': 0.5, 'max_depth': 10, 'subsample': 0.8, 'lambda': 1.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}


{'min_child_weight': 10, 'eta': 0.15, 'colsample_bytree': 0.5, 'max_depth': 12, 'subsample': 0.9, 'lambda': 0.5, 'nthread': -1, 'booster': 'gbtree', 'eval_metric': 'rmse', 'silent': 1, 'objective': 'reg:linear'}
train-rmse:0.933942 Modeling RMSLE 0.93526
